# Feature importance analysis
for Tree Mortality Prediction based on Growth Patterns 
using Machine Learning Models  
  
__Iulia Iordanescu__, Acton Boxborough Regional HS, 24iordanescui@abschools.org  
__Michael Dietze__, Department of Earth & Environment, Boston University, dietze@bu.edu  

### Harvard Forest Ecology Symposium
#### March 16-17, 2021  
   
   
data: [HF213](https://harvardforest1.fas.harvard.edu/exist/apps/datasets/showData.html?id=HF213)

Use classification algorithms to predict A(live) or D(ead) labels in __mortality13__ and __mortality14__ columns using these features:  
 - spp: USDA Plants database species code  
 - dbh09: diameter at Breast Height (1.4m) in year 2009 (unit: centimeter / missing value: NA)  
 - dbh11: diameter at Breast Height (1.4m) in year 2011 (unit: centimeter / missing value: NA)  
 - dbh12: diameter at Breast Height (1.4m) in year 2012 (unit: centimeter / missing value: NA)  
 - dbh13: diameter at Breast Height (1.4m) in year 2013 (unit: centimeter / missing value: NA)  
   
   
   
   
##### Tree mortality prediction based on growth patterns using Machine Learning
Feature importance analysis

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import sys, os, pathlib, shutil, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, RFECV

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    roc_curve,
    auc,
)
from sklearn.model_selection import (
    train_test_split, 
    StratifiedShuffleSplit,
    StratifiedKFold,
)

from sklearn.preprocessing import OrdinalEncoder

from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
# !pip install xgboost
import xgboost
print(xgboost.__version__)

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
MINIMUM_COUNT = 10
TRAIN_DATA = 0.6

In [ ]:
# !/opt/conda/bin/conda install -c anaconda seaborn pandas scikit-learn -y 

In [ ]:
%matplotlib inline

In [ ]:
!pwd
# !ls -la ./../data/hrvardf/HF213

In [ ]:
dataFileName='hf213-01-hf-inventory.csv'
dataPathFull= pathlib.Path('./../data/harvardf/HF213') / dataFileName
myData = pd.read_csv(str(dataPathFull)) 

In [ ]:
myData.shape
myData.head(2)
myData.tail(2)

In [ ]:
myData.info()
myData.isnull().sum()

In [ ]:
# myData.dropna()
# myData.isna().sum()


In [ ]:
# basic descriptive statistics for numeric columns:
myData.describe()

In [ ]:
corr=myData.corr()
plt.figure(figsize = (9, 7))
sns.heatmap(corr, cmap="RdBu",
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
# myData.groupby('spp').size()
myCols = ['spp', 'mortality14', 'dmg14']
myData[myCols[0]].value_counts(dropna=False) 
myData[myCols[1]].value_counts(dropna=False)
myData[myCols[2]].value_counts(dropna=False)
# myData.pivot_table(index = [myCols[0]]
#                    , columns = myCols[1]
#                    , values =  myCols[2]
#                    , aggfunc=np.sum, fill_value=0)
          


In [ ]:
import seaborn as sns
sns.countplot(x= myData['spp'],label="spp Count")
plt.show()

In [ ]:
myData['spp'].value_counts(dropna=False) 
removeSPP = myData['spp'].value_counts(dropna=False).loc[lambda x : x<MINIMUM_COUNT].index.tolist()
removeSPP

# filteredData = myData.replace(dict.fromkeys(removeSPP, 'TooFew'))
# filteredData['spp'].value_counts(dropna=False)

In [ ]:
featureColumn_01=['spp', 'dbh09', 'dbh11', 'dbh12']
# featureColumn_01=[ 'dbh09', 'dbh11', 'dbh12']
labelColumn_01 = 'mortality13'
featureColumn_02=['spp', 'dbh09', 'dbh11', 'dbh12', 'dbh13']
# featureColumn_02=['dbh09', 'dbh11', 'dbh12', 'dbh13']
labelColumn_02 = 'mortality14' 

labelColumn = labelColumn_02
featureColumn = featureColumn_02

In [ ]:
sorted(set(featureColumn+[labelColumn]))
filteredDataML = myData[sorted(set(featureColumn+[labelColumn]))]

# filteredDataML.shape
# filteredDataML.head()
# filteredDataML[labelColumn].value_counts(dropna=False)

In [ ]:
sns.pairplot(filteredDataML)

In [ ]:
from sklearn.impute import SimpleImputer
features_cols = [x for x in featureColumn if x != 'spp']
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(filteredDataML[features_cols])
imputed = imputer.transform(filteredDataML[features_cols])
filteredDataML[features_cols]=imputer.transform(filteredDataML[features_cols])

In [ ]:
filteredDataML.columns

In [ ]:
from  itertools import combinations
list(combinations(sorted(features_cols), 2))
[print(col1, col2) for col1, col2 in combinations(sorted(features_cols), 2)]

In [ ]:
filteredDataML.columns
# col1= trainData.columns[0]
# col2= trainData.columns[1]

# trainData1 = trainData.join(pd.DataFrame(((trainData[col2]/(trainData[col1]) -1)*100).rename(f'{col1}to{col2}')))

filteredDataML = filteredDataML.join(
    pd.concat([(((filteredDataML[col2]/(filteredDataML[col1]) -1)*100)
                .rename(f'{col2}to{col1}'))
               for col1, col2 in combinations(sorted(features_cols), 2)], 1))
# testData = testData.join(
#     pd.concat([(((testData[col2]/(testData[col1]) -1)*100)
#                 .rename(f'{col2}to{col1}'))
#                for col1, col2 in combinations(sorted(numCols), 2)], 1))
# validationData = validationData.join(
#     pd.concat([(((validationData[col2]/(validationData[col1]) -1)*100)
#                 .rename(f'{col2}to{col1}'))
#                for col1, col2 in combinations(sorted(numCols), 2)], 1))

# trainData.head(2)
# testData.head(2)
# validationData.head(2)
filteredDataML.columns

In [ ]:
mortality_13_14 = pd.crosstab(index=myData['mortality14']
                            ,columns=myData['mortality13']
                            , margins=True)
mortality_13_14

mortality_spp = pd.crosstab(index=myData['spp']
                            ,columns=myData['mortality14']
                            , margins=True)
mortality_spp


In [ ]:
corr=filteredDataML.corr()
corr
plt.figure(figsize = (9, 7))
sns.heatmap(corr, cmap="RdBu",
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
filteredDataML[labelColumn].value_counts(dropna=False)

filteredDataML = filteredDataML[filteredDataML[labelColumn].isin(['D', 'A'])]
filteredDataML.shape
filteredDataML.head()


filteredDataML[labelColumn].value_counts(dropna=False)

In [ ]:
# oh_encoder = OneHotEncoder(handle_unknown='ignore')
# enc_lbl_df = pd.DataFrame((oh_encoder.fit_transform(filteredDataML[['spp']])).toarray())
# enc_lbl_df


In [ ]:
# https://scikit-learn.org/stable/auto_examples/ensemble/plot_stack_predictors.html#sphx-glr-auto-examples-ensemble-plot-stack-predictors-py

catCols = filteredDataML.columns[filteredDataML.dtypes == 'O']
numCols = filteredDataML.columns[filteredDataML.dtypes == 'float64']
catCols
numCols


In [ ]:
print(filteredDataML.loc[0:3])

In [ ]:
stratifySplit = StratifiedShuffleSplit(n_splits=1, train_size=TRAIN_DATA, random_state=1)

trainIdx, tstIdx = next(stratifySplit.split(filteredDataML, filteredDataML[labelColumn]))
# print("\n Train:", sorted(trainIdx))
len(trainIdx)
len(tstIdx)

filteredDataML.loc[filteredDataML.index.intersection(filteredDataML.index[trainIdx])].shape
filteredDataML[filteredDataML.index.isin(filteredDataML.index[trainIdx])].shape
aa=filteredDataML.loc[filteredDataML.index.intersection(filteredDataML.index[tstIdx])]
aa.shape
stratifySplit = StratifiedShuffleSplit(n_splits=1, train_size=TRAIN_DATA, test_size=1-TRAIN_DATA, random_state=1)
testIdx, validationIdx = next(stratifySplit.split(aa,  aa[labelColumn]))

len(testIdx)
len(validationIdx)
filteredDataML.shape

# testIdx=tstIdx[testIdx]
# validationIdx=tstIdx[validationIdx]

# print("\n Test:", sorted(testIdx))
# print("\nValidation:", sorted(validationIdx))

In [ ]:
trainData=filteredDataML.loc[filteredDataML.index.intersection(filteredDataML.index[trainIdx]),:]
testData=aa.loc[aa.index.intersection(aa.index[testIdx]),:]
validationData = aa.loc[aa.index.intersection(aa.index[validationIdx]),:]

filteredDataML[labelColumn].value_counts(dropna=False)
trainData[labelColumn].value_counts(dropna=False) 
testData[labelColumn].value_counts(dropna=False) 
validationData[labelColumn].value_counts(dropna=False) 

In [ ]:
ordinalEncoder = OrdinalEncoder()
ordinalEncoder.fit(filteredDataML[catCols])
ordinalEncoder.categories_

trainData[catCols] = ordinalEncoder.transform(trainData[catCols])
testData[catCols] = ordinalEncoder.transform(testData[catCols])
validationData[catCols] = ordinalEncoder.transform(validationData[catCols])

trainData.head()

In [ ]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(missing_values=np.nan, strategy='median')
# imputer.fit(trainData[featureColumn])


In [ ]:
# trainData[featureColumn] = imputer.transform(trainData[featureColumn])
# testData[featureColumn] = imputer.transform(testData[featureColumn])
# validationData[featureColumn] = imputer.transform(validationData[featureColumn])

In [ ]:
# from  itertools import combinations
# trainData.columns
# # col1= trainData.columns[0]
# # col2= trainData.columns[1]

# # trainData1 = trainData.join(pd.DataFrame(((trainData[col2]/(trainData[col1]) -1)*100).rename(f'{col1}to{col2}')))

# trainData = trainData.join(
#     pd.concat([(((trainData[col2]/(trainData[col1]) -1)*100)
#                 .rename(f'{col2}to{col1}'))
#                for col1, col2 in combinations(sorted(numCols), 2)], 1))
# testData = testData.join(
#     pd.concat([(((testData[col2]/(testData[col1]) -1)*100)
#                 .rename(f'{col2}to{col1}'))
#                for col1, col2 in combinations(sorted(numCols), 2)], 1))
# validationData = validationData.join(
#     pd.concat([(((validationData[col2]/(validationData[col1]) -1)*100)
#                 .rename(f'{col2}to{col1}'))
#                for col1, col2 in combinations(sorted(numCols), 2)], 1))

# trainData.head(2)
# testData.head(2)
# validationData.head(2)
# trainData.columns


In [ ]:
newNumCols = trainData.columns[trainData.dtypes == 'float64']
newNumCols = list(newNumCols)
newNumCols.remove(labelColumn)

featureColumn = newNumCols

newNumCols

In [ ]:
from sklearn.preprocessing import RobustScaler
robustScaler = RobustScaler(quantile_range = (0.1,0.9))
robustScaler = robustScaler.fit(trainData[newNumCols])
trainData[newNumCols] =robustScaler.transform(trainData[newNumCols])
testData[newNumCols] =robustScaler.transform(testData[newNumCols])
validationData[newNumCols] =robustScaler.transform(validationData[newNumCols])

In [ ]:
trainData.describe()
testData.describe()
validationData.describe()

In [ ]:
# logistic regression for feature importance

# LogRegModel = LogisticRegression()
# LogRegModel.fit(trainData[featureColumn], trainData[labelColumn])

# # get importance
# importance = LogRegModel.coef_[0]
# # summarize feature importance
# print(featureColumn)
# for i,v in enumerate(importance):
# 	print('Feature: %0d, %s, Score: %.5f' % (i,featureColumn[i], v))
    
# # plot feature importance
# plt.bar([x for x in range(len(importance))], importance)
# plt.show()

### Feature importance analysis
for Tree Mortality Prediction based on Growth Patterns 
using Machine Learning Models  


In [ ]:
# CART Feature Importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


DecTreeModel = DecisionTreeClassifier()
RFModel = RandomForestClassifier()
XGBModel = XGBClassifier()


def FI_Analysis(crt_model):
    crt_model.fit(trainData[featureColumn], trainData[labelColumn])

    # get importance
    importance = crt_model.feature_importances_
    # summarize feature importance
    
    print(type(crt_model).__name__)
    for i,v in enumerate(importance):
        print('Feature: %0d, %s, Score: %.5f' % (i,featureColumn[i], v))

    # plot feature importance
    plt.bar([x for x in range(len(importance))], importance)
    plt.show()
    return importance

print(featureColumn)    
importances = [FI_Analysis(crt_model) for crt_model in [DecTreeModel, RFModel, XGBModel]]    

In [ ]:
model_names = [type(crt_model).__name__ for crt_model in [DecTreeModel, RFModel, XGBModel]] 
imps = pd.DataFrame.from_dict(dict(zip(model_names, importances))).set_index([featureColumn])

imps

# plt.pcolor(imps)
# plt.yticks(np.arange(0.5, len(imps.index), 1), imps.index)
# plt.xticks(np.arange(0.5, len(imps.columns), 1), imps.columns)
# plt.show()
sns.heatmap(imps, annot=True)

In [ ]:
from sklearn.inspection import permutation_importance

from sklearn.metrics import cohen_kappa_score, roc_auc_score, make_scorer

roc_auc_scorer = make_scorer(roc_auc_score)

SVC_model = svm.SVC(kernel='rbf', random_state=0, gamma=.1, C=100, probability=True)
KNN_model = KNeighborsClassifier(n_neighbors=50)

def perm_FI(crt_model, crt_scorer=roc_auc_scorer):

    crt_model.fit(trainData[featureColumn], trainData[labelColumn])
    
    # perform permutation importance
    results = permutation_importance(crt_model, trainData[featureColumn], trainData[labelColumn], scoring=crt_scorer)
    
    # get importance
    importance = results.importances_mean
    
    print(type(crt_model).__name__)
    # summarize feature importance
    for i,v in enumerate(importance):
        print('Feature: %0d, %s, Score: %.5f' % (i,featureColumn[i], v))


    # plot feature importance
    plt.bar([x for x in range(len(importance))], importance)
    plt.show()
    return importance

print(featureColumn)   
model_list = [DecTreeModel, RFModel, XGBModel, SVC_model]
importances = [perm_FI(crt_model) for crt_model in model_list]



In [ ]:
model_names = [type(crt_model).__name__ for crt_model in model_list] 
imps= pd.DataFrame.from_dict(dict(zip(model_names, importances))).set_index([featureColumn])

imps

sns.heatmap(imps, annot=True)

In [ ]:
from sklearn.metrics import cohen_kappa_score, roc_auc_score, make_scorer

cohen_kappa_scorer = make_scorer(cohen_kappa_score)
model_list = [DecTreeModel, RFModel, XGBModel, SVC_model]
importances = [perm_FI(crt_model, cohen_kappa_scorer) for crt_model in model_list]



In [ ]:
model_names = [type(crt_model).__name__ for crt_model in model_list] 
imps= pd.DataFrame.from_dict(dict(zip(model_names, importances))).set_index([featureColumn])

imps

sns.heatmap(imps, annot=True)